In [ ]:
# %matplotlib widget
import pickle
import numpy as np

method = "LN_COBYLA (NLopt)"
problem = "maxcut"
p = 5
n = 12
seed_pool = list(range(1000))

data = pickle.load(
    open(
        f"data/{problem}/configs/max_ar/LN_COBYLA (NLopt)-p{p}-q{n}-s0-999.pckl",
        "rb",
    ),
)
max_ar = data["result"][:len(seed_pool)]

data = pickle.load(
    open(
        f"data/{problem}/configs/budget/{method}-p{p}-q{n}-s{seed_pool[0]}-{seed_pool[-1]}.pckl",
        "rb",
    ),
)
config = data["config"]
result = data["result"][:,:,:]
initial_ar = np.array(data["initial_ar"])

print(config, result.shape)
shots_pool = [d["shots"] for d in config["executor_kwargs"]][:]
maxfev_pool = config["maxeval"][:]


In [ ]:
import networkx as nx
from qokit.parameter_utils import get_fixed_gamma_beta

delta = 0.01
p = 1
g = nx.random_regular_graph(3, n)
eval_func = get_evaluate_energy(
    g,
    None,
    p,
    objective="expectation",
    simulator="c",
)

gamma, beta = get_fixed_gamma_beta(3, p)
beta = [b * 4 for b in beta]
initial_value = eval_func(gamma + beta)
gamma_values, beta_values = [], []
for i in range(p):
    gamma[i] -= delta
    gamma_values.append(eval_func(gamma + beta))
    gamma[i] += delta
    beta[i] -= delta
    beta_values.append(eval_func(gamma + beta))
    beta[i] += delta
unweighted = (np.array(gamma_values) - initial_value), (np.array(beta_values) - initial_value)
print(unweighted)
np.mean(unweighted)

In [ ]:
from evaluate_energy import get_evaluate_energy, load_problem
import matplotlib.pyplot as plt
from scipy.spatial import distance

delta = 0.1
# seed_pool = [596, 617, 539, 972, 816, 0, 1, 2, 3, 4]
data = []
for seed in seed_pool:
    instance, precomputed_energies = load_problem(problem, n, seed)
    eval_func = get_evaluate_energy(
        instance,
        precomputed_energies,
        p,
        objective="expectation",
        simulator="c",
    )
    gamma, beta = get_fixed_gamma_beta(3, p)
    beta = [b * 4 for b in beta]
    initial_value = eval_func(gamma + beta)
    gamma_values, beta_values = [], []
    for i in range(p):
        gamma[i] += delta
        gamma_values.append(eval_func(gamma + beta))
        gamma[i] -= delta
        beta[i] += delta
        beta_values.append(eval_func(gamma + beta))
        beta[i] -= delta
    data.append((np.array(gamma_values) - initial_value) / (np.array(beta_values) - initial_value))
    print(data[-1], (np.array(gamma_values) - initial_value) / (np.array(beta_values) - initial_value), (max_ar - initial_ar)[seed])
    # data.append(distance.minkowski([instance.edges[e]['weight'] for e in instance.edges], np.ones(instance.number_of_edges())))

# plt.scatter((max_ar - initial_ar).flat, np.abs(np.array(data) - data[492]))
# plt.scatter((max_ar - initial_ar).flat, np.abs(data))
# plt.ylim(0, 1)
print(data[492], (max_ar - initial_ar)[492])

In [ ]:
data[404]

In [ ]:
# %matplotlib widget
import pickle
import numpy as np
import matplotlib.pyplot as plt

method = "LN_COBYLA (NLopt)"
problem = "maxcut"
depth_pool = list(range(1, 6))
depth_pool = [1, 5]
n = 10
seed_pool = list(range(1000))

max_ar, initial_ar = [], []
for p in depth_pool:
    data = pickle.load(
        open(
            f"data/{problem}/configs/max_ar/LN_COBYLA (NLopt)-p{p}-q{n}-s0-999.pckl",
            "rb",
        ),
    )
    max_ar.append(data["result"][:len(seed_pool)])
    data = pickle.load(
        open(
            f"data/{problem}/configs/budget/{method}-p{p}-q{n}-s{seed_pool[0]}-{seed_pool[-1]}.pckl",
            "rb",
        ),
    )
    initial_ar.append(np.array(data["initial_ar"]))

print(np.array(max_ar).shape)
plt.figure()
plt.violinplot(np.array(max_ar) - np.array(initial_ar))
